In [1]:
import torch 
from torch import utils
from torch import nn
import torch.nn.functional as F
from torch import optim
import pytorch_lightning as pl

import pandas as pd
import numpy as np

import pandas as pd

import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = np.load("data/data.npz", allow_pickle=True)

train_data = data["train"]
test_data = data['test']

train_data.shape, test_data.shape

((5000, 2), (3000,))

In [3]:
class OPdataset(utils.data.Dataset):
    def __init__(self, data) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, i):
        input, target = self.data[i]
        return torch.tensor(input), torch.tensor(target)


In [4]:
from sklearn.utils import shuffle


dataset = OPdataset(train_data)
train_dataset, eval_dataset = utils.data.random_split(dataset, 
                                                    (int(len(dataset)*0.75), 
                                                    int(len(dataset)*0.25)))

train_dataloader = utils.data.DataLoader(train_dataset, shuffle=True, batch_size=64)
eval_dataloader = utils.data.DataLoader(eval_dataset, shuffle=True, batch_size=64)

In [58]:
class ConvModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.vector_processor = nn.Sequential(
            nn.Linear(6,6),
            nn.GELU(),
            nn.Linear(6, 16),
            nn.GELU(),
            nn.Linear(16, 16),
            nn.GELU(),
            nn.Linear(16, 16),
            nn.GELU(),
            nn.Linear(16, 16),
            nn.GELU(),
            nn.Linear(16, 1),
            nn.GELU(),
        )
        self.points_computer = nn.Sequential(
            nn.Linear(300, 300),
            nn.GELU(),
            nn.Linear(300, 300),
            nn.GELU(),
            nn.Linear(300, 150),
            nn.GELU(),
            nn.Linear(150, 150),
            nn.GELU(),
            nn.Linear(150, 75),
            nn.GELU(),
            nn.Linear(75, 2),
        )
        self.loss_f = nn.CrossEntropyLoss()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.cat([x, torch.sin(x[:, :, :2])], dim=2)
        z = self.vector_processor(x)
        z = z.view(x.shape[0],-1)
        z = self.points_computer(z)

        return self.softmax(z)

    def configure_optimizers(self):
        return optim.Adam(self.parameters())

    def training_step(self, batch):
        x, y = batch
        z = self.forward(x.float())
        loss = self.loss_f(z, y)
        self.log('training_loss', loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        z = self.forward(x.float())
        loss = self.loss_f(z, y)
        self.log('eval_loss', loss)

        return loss


In [59]:
model = ConvModel()
trainer = pl.Trainer(max_epochs=32)

trainer.fit(model, train_dataloader, eval_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name             | Type             | Params
------------------------------------------------------
0 | vector_processor | Sequential       | 987   
1 | points_computer  | Sequential       | 259 K 
2 | loss_f           | CrossEntropyLoss | 0     
3 | softmax          | Softmax          | 0     
------------------------------------------------------
260 K     Trainable params
0         Non-trainable params
260 K     Total params
1.043     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/macos/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:489: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(
/Users/macos/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/macos/opt/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=32` reached.


In [60]:
x_test = torch.tensor(np.stack(test_data)).float()
test_result = model(x_test).argmax(axis=1).detach().numpy()

pd.DataFrame({'label': test_result}).to_csv('submission.csv',index=False)
